In [6]:
import pandas as pd
import duckdb
import argparse
import yaml
import os

In [2]:
conn = duckdb.connect()

In [3]:
# Specify the filename
filename9 = './data/input/Section111ValidICD9-Jan2024.csv'
filename10 = './data/input/Section111ValidICD10-Jan2024_edit.csv'

In [5]:

# Create a DuckDB table from the CSV file
conn.execute(f'CREATE TABLE valid_icd_9 AS SELECT * FROM read_csv_auto(\'{filename9}\', ALL_VARCHAR=1)')


In [6]:

# Create a DuckDB table from the CSV file
conn.execute(f'CREATE TABLE valid_icd_10 AS SELECT * FROM read_csv_auto(\'{filename10}\', ALL_VARCHAR=1)')

In [8]:

# Execute the SQL query and fetch the results
query = 'SELECT * FROM valid_icd_9 LIMIT 5'
result = conn.execute(query).fetchdf()


In [9]:
result

,CODE,LONG DESCRIPTION (VALID ICD-9 FY2024),NF EXCL
0,0010,Cholera due to vibrio cholerae,NaN
1,0011,Cholera due to vibrio cholerae el tor,NaN
2,0019,"Cholera, unspecified",NaN
3,0020,Typhoid fever,NaN
4,0021,Paratyphoid fever A,NaN


In [11]:

# Execute the SQL query and fetch the results
query2 = 'SELECT * FROM valid_icd_10 LIMIT 5'
result2 = conn.execute(query2).fetchdf()
result2

,column0,column1,column2,column3
0,CODE,SHORT DESCRIPTION (VALID ICD-10 FY2024),LONG DESCRIPTION (VALID ICD-10 FY2024),NF EXCL
1,A000,"Cholera due to Vibrio cholerae 01, biovar chol...","Cholera due to Vibrio cholerae 01, biovar chol...",NaN
2,A001,"Cholera due to Vibrio cholerae 01, biovar eltor","Cholera due to Vibrio cholerae 01, biovar eltor",NaN
3,A009,"Cholera, unspecified","Cholera, unspecified",NaN
4,A0100,"Typhoid fever, unspecified","Typhoid fever, unspecified",NaN


In [16]:

# Execute the SQL query and fetch the results
query = """
SELECT *
FROM valid_icd_10
WHERE column0 LIKE 'G20%'
   OR column0 LIKE 'G2111%'
ORDER BY column0"""
result = conn.execute(query).fetchdf()
result.column0

0      G20
1    G20A1
2    G20A2
3    G20B1
4    G20B2
5     G20C
6    G2111
Name: column0, dtype: object

In [ ]:
SELECT 
        * 
    FROM 
        read_csv_auto('icd10_valid.csv', header = TRUE)
    WHERE 
        SUBSTRING(icd10, 1, 3) >= 'I00' AND SUBSTRING(icd10, 1, 3) <= 'I99' OR
        SUBSTRING(icd10, 1, 4) >= 'Q200' AND SUBSTRING(icd10, 1, 4) <= 'Q280'


In [36]:

# Define the path to the YAML file
yaml_file_path = './conf/icd_codes/icd_codes_2.yml'

# Read YAML file into a dictionary
with open(yaml_file_path, 'r') as yaml_file:
    icd_codes = yaml.safe_load(yaml_file)


In [37]:

# Initialize empty dictionaries to store results
icd9_result_dict = {}
icd10_result_dict = {}

# Loop through each section in icd_codes dictionary
for key, values in icd_codes.items():
    icd9_result_list = []
    icd10_result_list = []

    for icd9_code in values["icd9"]:
        query = f"""
        SELECT *
        FROM valid_icd_9
        WHERE CODE LIKE '{icd9_code}%'
        ORDER BY CODE
        """
        result = conn.execute(query).fetchdf()
        icd9_result_list.extend(result.CODE.tolist())

    for icd10_code in values["icd10"]:
        query = f"""
        SELECT *
        FROM valid_icd_10
        WHERE column0 LIKE '{icd10_code}%'
        ORDER BY column0
        """
        result = conn.execute(query).fetchdf()
        icd10_result_list.extend(result.column0.tolist())

    # Add icd9 and icd10 result lists to respective dictionaries
    icd9_result_dict[key] = {"icd9": icd9_result_list}
    icd10_result_dict[key] = {"icd10": icd10_result_list}


In [38]:
icd9_result_dict

{'pd': {'icd9': ['3320', '3321']},
 'dementia': {'icd9': ['29410', '29411']},
 'lewy': {'icd9': ['33182']}}

In [39]:
icd10_result_dict

{'pd': {'icd10': ['G20',
   'G20A1',
   'G20A2',
   'G20B1',
   'G20B2',
   'G20C',
   'G2111',
   'G2119',
   'G213',
   'G214',
   'G218',
   'G219']},
 'dementia': {'icd10': ['F0280', 'F0281', 'F02811', 'F02818']},
 'lewy': {'icd10': ['G3183']}}

In [7]:
%%bash
python get_outcomes.py --year 2015


## Preparing outcomes ----
['pd', 'dementia', 'lewy']
preparing pd ----
## Preparing query----

        SELECT bene_id, adm_id 
        FROM (SELECT bene_id, adm_id, UNNEST(diagnoses) AS diag FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2015.parquet')
        WHERE diag IN ('3320','3321','G20','G20A1','G20A2','G20B1','G20B2','G20C','G2111','G2119','G213','G214','G218','G219')
        


Traceback (most recent call last):
  File "/n/dominici_nsaph_l3/Lab/data_processing/medpar_outcomes/kezia/get_outcomes.py", line 108, in <module>
    main(args)
  File "/n/dominici_nsaph_l3/Lab/data_processing/medpar_outcomes/kezia/get_outcomes.py", line 61, in main
    outcome_df = conn.execute(query).fetchdf()
RuntimeError: IO Error: No files found that match the pattern "./data/input/mbsf_medpar_denom/medpar_hospitalizations_2015.parquet"


CalledProcessError: Command 'b'python get_outcomes.py --year 2015\n'' returned non-zero exit status 1.

In [8]:
query = """SELECT bene_id, adm_id 
FROM (SELECT bene_id, adm_id, UNNEST(diagnoses) AS diag FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2015.parquet')
WHERE diag IN ('3320','3321','G20','G20A1','G20A2','G20B1','G20B2','G20C','G2111','G2119','G213','G214','G218','G219')
"""

In [9]:
outcome_df = conn.execute(query).fetchdf()

RuntimeError: IO Error: No files found that match the pattern "./data/input/mbsf_medpar_denom/medpar_hospitalizations_2015.parquet"

In [10]:
%%bash

for year in {2000..2016}
do
    python get_outcomes.py --year $year
done

## Preparing outcomes ----
['pd', 'dementia', 'lewy']
preparing pd ----
## Preparing query----

        SELECT bene_id, adm_id 
        FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2000.parquet'
        WHERE diagnoses[1] IN ('3320','3321','G20','G20A1','G20A2','G20B1','G20B2','G20C','G2111','G2119','G213','G214','G218','G219')
        
(12557, 3)
           bene_id                            adm_id outcome
0  lllllllo4Uo7lo4  178e31fa8847b4b72a8b6dfdf8d1aabb      pd
1  lllllllo4l808SS  43e16db50395934e6729c01f503d004f      pd
2  llllllllUOo4U84  26f1e4351a818b171157c600eb0286f1      pd
3  llllllloX7004XU  37253f45709f5df71623ef351acf4bab      pd
4        A07731474  1cd4335fd49fe37e4cdb3877f74d8655      pd
preparing dementia ----
## Preparing query----

        SELECT bene_id, adm_id 
        FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2000.parquet'
        WHERE diagnoses[1] IN ('29410','29411','F0280','F0281','F02811','F02818')
        
(692, 3)
     

Traceback (most recent call last):
  File "/n/dominici_nsaph_l3/Lab/data_processing/medpar_outcomes/kezia/get_outcomes.py", line 108, in <module>
    main(args)
  File "/n/dominici_nsaph_l3/Lab/data_processing/medpar_outcomes/kezia/get_outcomes.py", line 61, in main
    outcome_df = conn.execute(query).fetchdf()
RuntimeError: IO Error: No files found that match the pattern "./data/input/mbsf_medpar_denom/medpar_hospitalizations_2015.parquet"


## Preparing outcomes ----
['pd', 'dementia', 'lewy']
preparing pd ----
## Preparing query----

        SELECT bene_id, adm_id 
        FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2016.parquet'
        WHERE diagnoses[1] IN ('3320','3321','G20','G20A1','G20A2','G20B1','G20B2','G20C','G2111','G2119','G213','G214','G218','G219')
        
(24259, 3)
           bene_id                            adm_id outcome
0  llllllllolU0UUU  f6162c1b5fee2cbd0f3bcfb05c3ceb59      pd
1  llllllllolU0UUU  5a42f8d1e0f566d9f564fe89cb6df240      pd
2  lllllllOlO0U4Sl  ce4db0182c9bc87a2282dd791dbac6f8      pd
3  llllllU7SOXU77l  daa285740457964729ada7aa2b6a29ff      pd
4  llllllU7SOXU77l  bdfdc5eea841c5218d5cae4b7350f1dc      pd
preparing dementia ----
## Preparing query----

        SELECT bene_id, adm_id 
        FROM './data/input/mbsf_medpar_denom/medpar_hospitalizations_2016.parquet'
        WHERE diagnoses[1] IN ('29410','29411','F0280','F0281','F02811','F02818')
        
(3545, 3)
    